In [1]:
import glob,os
import sys
import pandas as pd
import string
import numpy as np
from tqdm import tqdm
from time import sleep
import sys
from gensim.corpora import WikiCorpus
from multiprocessing import Pool
import json
import pickle
from collections import defaultdict
import gc
import torch
from scipy import sparse

In [2]:
class Vocab:
    PAD_TOKEN = '<pad>'

    def __init__(self):
        self.w2i = {}
        self.i2w = []
        self.support = []
        self.add_token(Vocab.PAD_TOKEN)
        self.cached_neg_sample_prob = None

    def pad_id(self):
        return self.get_id(Vocab.PAD_TOKEN)

    def add_tokens(self, tokens):
        for tidx, token in enumerate(tokens):
            self.add_token(token)

    def add_token(self, token, token_support=1):
        if token not in self.w2i:
            self.w2i[token] = len(self.i2w)
            self.i2w.append(token)
            self.support.append(0)
        self.support[self.get_id(token)] += token_support

    def neg_sample(self, size=None):
        if self.cached_neg_sample_prob is None:
            support = np.array(self.support)
            support_raised = np.power(support, 0.75)
            support_raised[0] = 0.0  # Never select padding idx
            self.cached_neg_sample_prob = support_raised / support_raised.sum()
        return np.random.choice(np.arange(self.size()), size=size, p=self.cached_neg_sample_prob)

    def get_id(self, token):
        if token in self.w2i:
            return self.w2i[token]
        return -1

    def id_count(self, id):
        return self.support[id]

    def token_count(self, token):
        return self.id_count(self.get_id(token))

    def get_ids(self, tokens):
        return list(map(self.get_id, tokens))

    def get_token(self, id):
        return self.i2w[id]

    def size(self):
        return len(self.i2w)

In [3]:
class extract_data:

    def __init__(self,data_path,path):
        self.X = []
        self.vocab = []
        self.tokens = []
        self.keep_prob = []
        self.subsampled_ids = []
        self.tokenized_subsampled_data = []
        self.data_path = data_path
        self.path = path
        
    def make_corpus(self,in_f, out_f):
        print("Loading data")
        wiki = WikiCorpus(in_f)
        print("Processing...")
        i = 1
        for document in wiki.get_texts():
            out_f_ = ''.join([out_f,'\\wiki_en_',str(i),".txt"])
            output = open(out_f_, 'w',encoding='utf-8')
            output.write(' '.join(document))
            output.close()
            i+=1
            if i%10000 == 0:  
                print(i,"documents procecessed")
        print('Processing complete!')

    def read_data(self,keep_prob = 0.25):
            X = []
            print("Reading txt data")
            os.chdir(self.data_path)
            N = len(glob.glob("*.txt"))
            #choice = np.random.choice(N, sample_size, replace=False)+1
            for i in tqdm(range(1,N+1)):
                if np.random.binomial(1, keep_prob):
                    text = ''.join([self.data_path,"\\wiki_en_",str(i),".txt"])
                    with open(text,'r',encoding='utf-8') as f:
                        X.append(''.join(list(f)).lower())
                else:
                    pass
            print("Saving sampled wiki data, sample rate:",keep_prob)
            with open(self.path + 'Wiki_tokenized{}.json'.format(''), 'w') as fd:
                json.dump(list(zip(range(len(X)),X)), fd)
    
    def token_counts(self):
        print("Generating token counts")
        token_cts = defaultdict(int)
        doc_id = 0
        token_counts_fn = self.path + 'Wiki_tokenized{}.json'.format('')
        with open(token_counts_fn, 'r') as fd:
            X = json.load(fd)
        for doc in X:
            for token in doc[1].split():
                token_cts[token] += 1
                token_cts['__ALL__'] += 1
        print("Saving token counts")
        with open(self.path + 'Wiki_token_counts{}.json'.format(''), 'w') as fd:
            json.dump(token_cts, fd)
    
    def subsample(self):
        print("Sub-sampling tokens...")
        tokenized_fp = self.path + 'Wiki_tokenized'
        token_counts_fp = self.path + 'Wiki_token_counts'
        subsample_param = 0.001
        min_token_count = 5
        debug_str = ''
        tokenized_data_fn = '{}{}.json'.format(tokenized_fp, debug_str)
        with open(tokenized_data_fn, 'r') as fd:
            tokenized_data = json.load(fd)
        token_counts_fn = '{}{}.json'.format(token_counts_fp, debug_str)
        with open(token_counts_fn, 'r') as fd:
            token_counts = json.load(fd)
        N = float(token_counts['__ALL__'])
        # And vocabulary with word counts
        self.vocab = Vocab()
        num_docs = len(tokenized_data)
        for doc_idx in tqdm(range(num_docs)):
            category, tokenized_doc_str = tokenized_data[doc_idx]
            subsampled_doc = []
            for token in tokenized_doc_str.split():
                wc = token_counts[token]
                too_sparse = wc <= min_token_count
                if too_sparse:
                    continue
                frac = wc / N
                keep_prob = min((np.sqrt(frac / subsample_param) + 1) * (subsample_param / frac), 1.0)
                should_keep = np.random.binomial(1, keep_prob) == 1
                if should_keep:
                    subsampled_doc.append(token)
                    self.vocab.add_token(token, token_support=1)
            self.tokenized_subsampled_data.append((category-1, ' '.join(subsampled_doc)))
            
    def tokens_to_ids(self):
        print("Converting tokens to ids...")
        for i in tqdm(range(len(self.tokenized_subsampled_data))):
            self.subsampled_ids.append(np.asarray([self.vocab.get_id(x) for x in self.tokenized_subsampled_data[i][1].split()]))
        self.subsampled_ids = np.asarray(self.subsampled_ids)
    
    def token_doc_map(self):
        print("Forming token document matrix... ")
        self.token_doc_matrix = np.zeros((self.vocab.size()+1,self.subsampled_ids.shape[0]))
        for i in tqdm(range(self.subsampled_ids.shape[0])):
            for token_id in self.subsampled_ids[i]:
                self.token_doc_matrix[token_id][i] +=1
        self.token_doc_matrix = sparse.csr_matrix(self.token_doc_matrix)        

In [4]:
in_f = "D:\Latent Meaning Cells\simplewiki-latest-pages-articles.xml.bz2"
out_f = 'D:\Latent Meaning Cells\simplewiki'

data_path =  "D:\\Latent Meaning Cells\\simplewiki"
path = "D:\\Latent Meaning Cells\\"

wiki_data = extract_data(data_path,path)
#wiki_data.make_corpus(in_f, out_f)
try:
    p = Pool(processes=10)
    p.apply(wiki_data.read_data())
except:
    p.close()
    wiki_data.token_counts()
    wiki_data.subsample()
    wiki_data.tokens_to_ids()
    wiki_data.token_doc_map()

Reading txt data


100%|████████████████████████████████████████████████████████████████████████| 100527/100527 [00:26<00:00, 3805.78it/s]


Saving sampled wiki data, sample rate: 0.25
Generating token counts
Saving token counts
Sub-sampling tokens...


  2%|█▍                                                                          | 466/24903 [00:00<00:05, 4626.20it/s]

Converting tokens to ids...


  1%|▋                                                                           | 206/24903 [00:00<00:12, 2044.87it/s]

Forming token document matrix... 


100%|██████████████████████████████████████████████████████████████████████████| 24903/24903 [00:05<00:00, 4404.58it/s]


In [5]:
sub_tokens = wiki_data.subsampled_ids
vocab = wiki_data.vocab

In [24]:
#filename = "D:\Latent Meaning Cells\\vocab.obj" 
#file_pi = open(filename, 'wb')
#pickle.dump(vocab, file_pi)

If already extracted:

In [9]:
#filename = "D:\Latent Meaning Cells\sub_tokens.obj"
#filehandler = open(filename, 'rb')
#sub_tokens = pickle.load(filehandler)

In [10]:
#filename = "D:\Latent Meaning Cells\\vocab.obj"
#filehandler = open(filename, 'rb')
#vocab = pickle.load(filehandler)

In [6]:
class batcher:

    def __init__(self,batch_size,window_size,vocab,sub_tokens):

        self.batch_size = batch_size
        self.window_size = window_size
        self.vocab = vocab
        self.sub_tokens = sub_tokens
        
        self.vocab_tokens = np.linspace(1, vocab.size()-1, num=vocab.size()-1).astype(int)
        #self.prob = np.power(vocab.support[1:], 0.75)
        #self.prob = self.prob/np.sum(self.prob)
        
    def next(self):
        
        sub_tokens = self.sub_tokens
        batch_size = self.batch_size
        window_size = self.window_size
        #prob = self.prob
        
        center_words = np.zeros(batch_size)
        vocab_tokens =self.vocab_tokens
        num_contexts = np.zeros(batch_size)
        
        positive_words = np.zeros((batch_size,window_size*2))
        negative_words = np.zeros((batch_size,window_size*2))
        doc_ids = np.random.choice(len(sub_tokens),batch_size)
        len_docs = np.asarray([x.shape[0] for x in sub_tokens[doc_ids]])
        center_index = np.asarray([np.random.choice(x) for x in len_docs])
        upper_index = np.minimum(center_index+window_size,len_docs-1).astype(int)
        lower_index = np.maximum(center_index-window_size,np.zeros(batch_size)).astype(int)
            
        for i in range(batch_size):
        
            positive_sub_batch = np.linspace(lower_index[i],upper_index[i], num=upper_index[i]-lower_index[i]+1)
            positive_sub_batch = positive_sub_batch[positive_sub_batch != center_index[i]].astype(int)
            
            num_contexts[i] = positive_sub_batch.shape[0]
            
            document = sub_tokens[doc_ids[i]]
            positive_sub_batch = np.asarray([document[x] for x in positive_sub_batch]).astype(int)
            positive_words[i,:positive_sub_batch.shape[0]] = positive_sub_batch

            center_words[i] = document[center_index[i]]
            
            #negative_words_ = vocab_tokens[~np.isin(vocab_tokens, positive_sub_batch)]
            #negative_sampling_probability = prob[~np.isin(vocab_tokens, positive_sub_batch)]
            #negative_sampling_probability = negative_sampling_probability/np.sum(negative_sampling_probability)
            #negative_words[i] = np.random.choice(negative_words_, window_size*2, p=negative_sampling_probability).astype(int)
            
            negative_words[i] = vocab.neg_sample(window_size*2)
            
        return doc_ids.astype(int), center_words.astype(int), positive_words.astype(int), negative_words.astype(int),num_contexts.astype(int)

In [10]:
def mask_2D(target_size, num_contexts):
    mask = torch.BoolTensor(target_size)
    mask.fill_(0)
    for batch_idx, num_c in enumerate(num_contexts):
        if num_c < target_size[1]:
            mask[batch_idx, num_c:] = 1
    return mask

In [11]:
import numpy as np
import torch
import torch.nn as nn


class VAE(nn.Module):
    def __init__(self, device,encoder_input_dim,encoder_hidden_dim,latent_dim, token_vocab_size, section_vocab_size):
        super(VAE, self).__init__()
        self.device = device
        self.encoder = Encoder(encoder_input_dim,encoder_hidden_dim,latent_dim, token_vocab_size, section_vocab_size)
        self.margin = 1.0

    def forward(self, center_ids, section_ids, context_ids, neg_context_ids,num_context_ids):
        """
        :param center_ids: batch_size
        :param section_ids: batch_size
        :param context_ids: batch_size, 2 * context_window
        :param neg_context_ids: batch_size, 2 * context_window
        :param num_contexts: batch_size (how many context words for each center id - necessary for masking padding)
        :return: cost components: KL-Divergence (q(z|w,c) || p(z|w)) and max margin (reconstruction error)
        """
        # Mask padded context ids
        batch_size, num_context_ids = context_ids.size()
        mask_size = torch.Size([batch_size, num_context_ids])
        mask = mask_2D(mask_size, num_contexts).to(self.device)

        # Compute center words
        mu_center, sigma_center = self.encoder(center_ids, section_ids)
        mu_center_tiled = mu_center.unsqueeze(1).repeat(1, num_context_ids, 1)
        sigma_center_tiled = sigma_center.unsqueeze(1).repeat(1, num_context_ids, 1)
        mu_center_flat = mu_center_tiled.view(batch_size * num_context_ids, -1)
        sigma_center_flat = sigma_center_tiled.view(batch_size * num_context_ids, -1)

        # Tile section ids for positive and negative samples
        section_ids_tiled = section_ids.unsqueeze(-1).repeat(1, num_context_ids)

        # Compute positive and negative encoded samples
        mu_pos_context, sigma_pos_context = self.encoder(context_ids, section_ids_tiled)
        mu_neg_context, sigma_neg_context = self.encoder(neg_context_ids, section_ids_tiled)

        # Flatten positive context
        mu_pos_context_flat = mu_pos_context.view(batch_size * num_context_ids, -1)
        sigma_pos_context_flat = sigma_pos_context.view(batch_size * num_context_ids, -1)

        # Flatten negative context
        mu_neg_context_flat = mu_neg_context.view(batch_size * num_context_ids, -1)
        sigma_neg_context_flat = sigma_neg_context.view(batch_size * num_context_ids, -1)

        # Compute KL-divergence between center words and negative and reshape
        kl_pos_flat = compute_kl(mu_center_flat, sigma_center_flat, mu_pos_context_flat, sigma_pos_context_flat)
        kl_neg_flat = compute_kl(mu_center_flat, sigma_center_flat, mu_neg_context_flat, sigma_neg_context_flat)
        kl_pos = kl_pos_flat.view(batch_size, num_context_ids)
        kl_neg = kl_neg_flat.view(batch_size, num_context_ids)

        hinge_loss = (kl_pos - kl_neg + self.margin).clamp_min_(0)
        hinge_loss = hinge_loss.masked_fill(mask, 0)
        hinge_loss = hinge_loss.sum(1)
        return hinge_loss.mean()

In [12]:
import numpy as np
import torch
from torch import nn
from torch.nn import functional as F
import torch.utils.data
 
class Encoder(nn.Module):
    def __init__(self, encoder_input_dim,encoder_hidden_dim,latent_dim, token_vocab_size, section_vocab_size):
        super(Encoder, self).__init__()
        self.dropout = nn.Dropout(0.2)
        self.f = nn.Linear(encoder_input_dim * 2, encoder_hidden_dim, bias=True)
        self.u = nn.Linear(encoder_hidden_dim, latent_dim, bias=True)
        self.v = nn.Linear(encoder_hidden_dim, 1, bias=True)
    
        self.token_embeddings = nn.Embedding(token_vocab_size, encoder_input_dim, padding_idx=0)
        #self.token_embeddings.weight.data.uniform_(-1, 1)
        self.section_embeddings = nn.Embedding(section_vocab_size, encoder_input_dim)
        #self.section_embeddings.weight.data.uniform_(-1, 1)
        
    def forward(self, center_ids, section_ids):
        """
        :param center_ids: LongTensor of batch_size
        :param context_ids: LongTensor of batch_size
        :param mask: BoolTensor of batch_size x 2 * context_window (which context_ids are just the padding idx)
        :return: mu (batch_size, latent_dim), logvar (batch_size, 1)
        """
        center_embedding = self.token_embeddings(center_ids)
        section_embedding = self.section_embeddings(section_ids)
            
        merged_embeds = self.dropout(torch.cat([center_embedding, section_embedding], dim=-1))
            
        h = self.dropout(F.relu(self.f(merged_embeds)))
        var_clamped = self.v(h).exp().clamp_min(1.0)
        return self.u(h), var_clamped

In [13]:
def compute_kl(mu_a, sigma_a, mu_b, sigma_b, device=None):
    """
    :param mu_a: mean vector of batch_size x dim
    :param sigma_a: standard deviation of batch_size x {1, dim}
    :param mu_b: mean vector of batch_size x dim
    :param sigma_b: standard deviation of batch_size x {1, dim}
    :return: computes KL-Divergence between 2 diagonal Gaussian (a||b)
    """
    var_dim = sigma_a.size()[-1]
    assert sigma_b.size()[-1] == var_dim
    if var_dim == 1:
        return kl_spher(mu_a, sigma_a, mu_b, sigma_b)
    return kl_diag(mu_a, sigma_a, mu_b, sigma_b, device=device)

In [14]:
def kl_spher(mu_a, sigma_a, mu_b, sigma_b):
    """
    :param mu_a: mean vector of batch_size x dim
    :param sigma_a: standard deviation of batch_size x 1
    :param mu_b: mean vector of batch_size x dim
    :param sigma_b: standard deviation of batch_size x 1
    :return: computes KL-Divergence between 2 spherical Gaussian (a||b)
    """
    d = mu_a.shape[1]
    sigma_p_inv = 1.0 / sigma_b  # because diagonal
    tra = d * sigma_a * sigma_p_inv
    quadr = sigma_p_inv * torch.pow(mu_b - mu_a, 2).sum(1, keepdim=True)
    log_det = - d * torch.log(sigma_a * sigma_p_inv)
    res = 0.5 * (tra + quadr - d + log_det)
    return res

In [26]:
device="cuda"
encoder_input_dim = 100
encoder_hidden_dim = 64
latent_dim = 100

token_vocab_size = vocab.size()
section_vocab_size = sub_tokens.shape[0]

model = VAE(device,encoder_input_dim,encoder_hidden_dim,latent_dim, token_vocab_size, section_vocab_size).to(device)

trainable_params = filter(lambda x: x.requires_grad, model.parameters())
optimizer = torch.optim.Adam(trainable_params, lr=0.1)
optimizer.zero_grad()

In [ ]:
window_size = 5
batch_size = 1024
num_epoch = 70
num_contexts = batch_size
generator = batcher(batch_size,window_size,vocab,sub_tokens)


for epoch in range(1, num_epoch + 1):
    sleep(0.1)  # Make sure logging is synchronous with tqdm progress bar
    print('Starting Epoch={}'.format(epoch))
    generator = batcher(batch_size,window_size,vocab,sub_tokens)
    num_batches = batch_size
    epoch_loss = 0.0
    for _ in tqdm(range(int(vocab.size()/num_batches)), position=0, leave=True):
        # Reset gradients
        optimizer.zero_grad()

        section_ids,center_ids, context_ids, neg_ids,num_contexts = generator.next()
        
        center_ids_tens = torch.LongTensor(center_ids).to(device)
        context_ids_tens = torch.LongTensor(context_ids).to(device)
        section_ids_tens = torch.LongTensor(section_ids).to(device)
        neg_ids_tens = torch.LongTensor(neg_ids).to(device)

        loss = model(center_ids_tens, section_ids_tens, context_ids_tens, neg_ids_tens,num_contexts)
        loss.backward()  # backpropagate loss

        epoch_loss += loss.item()
        optimizer.step()
    sleep(0.1)
    print('Epoch={}. Loss={}.'.format(epoch, loss.item()))

  0%|                                                                                           | 0/44 [00:00<?, ?it/s]

Starting Epoch=1


  0%|                                                                                           | 0/44 [00:00<?, ?it/s]

Epoch=1. Loss=8.789356231689453.
Starting Epoch=2


  0%|                                                                                           | 0/44 [00:00<?, ?it/s]

...

  0%|                                                                                           | 0/44 [00:00<?, ?it/s]

Epoch=3. Loss=8.011993408203125.
Starting Epoch=4


  0%|                                                                                           | 0/44 [00:00<?, ?it/s]

Epoch=4. Loss=7.812597274780273.
Starting Epoch=5


 93%|████████████████████████████████████████████████████████████████████████████▍     | 41/44 [00:27<00:01,  1.52it/s]

In [309]:
dists = np.zeros(vocab.size())
month = model.encoder.token_embeddings(torch.Tensor(np.asarray([293])).long().to("cuda")).data.cpu().numpy()
for i in range(vocab.size()):
    diff = model.encoder.token_embeddings(torch.Tensor(np.asarray([i])).long().to("cuda")).data.to("cpu").numpy() - month
    dist = np.sum(np.dot(diff,diff.T))
    dists[i] = dist

Meaning distributions:

In [21]:
model.encoder.token_embeddings(torch.Tensor(np.asarray([0])).long().to("cuda")).data.cpu().numpy()

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],
      dtype=float32)